## Uncertainty = 1) Aleatoric Uncertainty + 2) Epistemic Uncertainty
- 1) Aleatoric Uncertainty (; due to "Data")
- 2) Epistemic Uncertainty (; due to "Model" + "often a lack of training data")

## Bayesian Deep Learning 

### MC Dropout -> Aleatoric Uncertainty 측정

- 1. Train Phase: Dropout "on"
- 2. Inference Phase: Dropout "on" (마지막 부근 layer의 dropout 1개 또는 2개 정도 "on" / "0.1 ~ 0.5")
- (만약 사용하는 모델의 마지막 부근 layer에 Dropout이 없으면, Dropout을 추가하고 Train 단계에서는 "off", Inference 단계에서 "on")
- 3. Inference number: 50 ~ 100회

## Heatmap

In [5]:
import tensorflow as tf 

from glob import glob 
import cv2 
import numpy as np 

import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [7]:
dl_model = tf.keras.models.load_model('C:/Users/user/Desktop/models/child_segmentation/20220725-102354_model.h5', custom_objects={'bce_jaccard_loss' : sm.losses.bce_jaccard_loss()})

TypeError: __call__() missing 2 required positional arguments: 'gt' and 'pr'

In [ ]:
""" 예측한 Segmentation 결과를 폴더에서 불러와서 Heatmap을 그려줌 """

inference_num = 50 ## inference 횟수
img_size = 400

test_pred_imgs = sorted(glob.glob(pred_dir + '/*.png'))
heatmap_dir = 'C:/Users/user/Desktop/models/child_segmentation/'


ni_list = []
for ni in range(1, int(len(test_pred_imgs)/inference_num)+1):
    ni_list.append(inference_num*ni-1)
    
    
heatmap_img_list = []    
heatmap_img = np.zeros((img_size,img_size), np.uint8)
for i in range(len(test_pred_imgs)):
    pred_img = cv2.imread(test_pred_imgs[i],0)  
    heatmap_img = np.add(heatmap_img, pred_img)
    
    for ni in ni_list:
        if i == ni:
            new_heatmap = 255-(heatmap_img/inference_num)
            heatmap_img_list.append(new_heatmap)
            heatmap_img = np.zeros((img_size,img_size), np.uint8)
                  
            """ Save """
            image.imsave(heatmap_dir + 'heatmap_' + str(test_pred_imgs[i][-10:-6]) + '.png', new_heatmap, cmap='OrRd')



## Evaluation Metrics

- Table1. Pixel acc, Mean accurcy, Dice Coefficient
- Table2. p(acc|certain), p(uncertain|inacc), PAvPU